In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [3]:
import math
import pickle
from tqdm import tqdm_notebook as tqdm

In [4]:
main_url = "https://www.zalando.co.uk/mens-clothing/"
base_url = "https://www.zalando.co.uk/"
base_path = "./zalando/pickles/"

In [20]:
# getting the list of all the categories along with sub-categories
html = urlopen(main_url)
soup = BeautifulSoup(html, 'lxml')

# getting the main categories
main_categories = []
anchors = soup.findAll("a", {"class": "cat_link-mTK6o"})
for anchor in anchors:
    main_categories.append(anchor.get("href").split("/")[1])
    
main_categories = main_categories[1:]
print("main categories")
print("________________________________________________")
print(main_categories)
print("------------------------------------------------")

# getting the sub categories
categories = {} # dictionary to store all the sub categories along with main categories
for category in main_categories:
    url = base_url + category +"/"
    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')
    anchors = soup.findAll(True, {"class": "cat_link-mTK6o cat_highlight-2DmxP"})[0].find_next('div').findAll("a", {"class": "cat_link-mTK6o"})
    category_list = []
    for anchor in anchors:
        category_list.append(anchor.get("href").split("/")[1])
    categories[category] = category_list

print("categories with sub categories")
print("_________________________________________________")
print(categories)
print("-------------------------------------------------")


main categories
________________________________________________
['mens-clothing-coats', 'mens-clothing-jackets', 'mens-clothing-jeans', 'mens-clothing-jumpers-cardigans', 'mens-clothing-shirts', 'mens-clothing-socks', 'mens-sports-clothing', 'mens-clothing-suits-ties', 'mens-clothing-swimwear', 'mens-clothing-t-shirts', 'men-clothing-tracksuits', 'mens-clothing-trousers-chinos', 'mens-clothing-underwear']
------------------------------------------------
categories with sub categories
_________________________________________________
{'mens-clothing-coats': ['mens-clothing-coats-winter-coats', 'mens-clothing-coats-wool-coats', 'mens-clothing-coats-down-coats', 'mens-clothing-coats-parkas', 'mens-clothing-coats-trench-coats', 'mens-clothing-coats-short-coats'], 'mens-clothing-jackets': ['mens-clothing-jackets-winter-jackets', 'mens-clothing-jackets-down-jackets', 'mens-clothing-jackets-leather-jackets', 'mens-waterproof-jackets', 'mens-clothing-jackets-athletic-jackets', 'mens-clothing-

In [8]:
# categories_old = dict(categories)

In [13]:
# selected_keys = ["mens-clothing-jeans", "mens-clothing-trousers-chinos"]

In [14]:
# categories = {key:value for key,value in categories_old.items() if key in selected_keys}

In [15]:
# categories

{'mens-clothing-jeans': ['mens-clothing-jeans-skinny-fit',
  'mens-clothing-jeans-slim-fit',
  'mens-clothing-jeans-straight-leg',
  'mens-clothing-jeans-loose-fit',
  'mens-clothing-jeans-bootcut',
  'mens-clothing-jeans-denim-shorts'],
 'mens-clothing-trousers-chinos': ['mens-clothing-chinos',
  'mens-clothing-joggers-sweats',
  'mens-clothing-trousers',
  'mens-clothing-cargo-trousers',
  'mens-clothing-shorts']}

In [21]:
# import statements for selenium

from selenium import webdriver

options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome('/home/tensor/Documents/softwares/chromedriver', chrome_options=options)

/home/tensor/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [18]:
# # Going throuhg all the sub categories of each categories to save all the image links
# file = open("images.txt", "w")
# count = 0
# for main_category in tqdm(categories):
#     for sub_category in tqdm(categories[main_category]):
#         url = base_url + sub_category
        
#         html = urlopen(url)
#         soup = BeautifulSoup(html, 'lxml')
        
#         # now getting the total number of pages for each category from total items, taking 84 items for each page
#         total_items = int(soup.find("div", {"class": "cat_count-1zSG1"}).get_text().split(" ")[0].replace(',', ''))
#         total_pages = math.ceil(total_items/84)
        
#         # now opening each of the pages in selenium because in beautiful soup all the images are not available
#         # and putting all the urls to as a list in pickle format
#         for i in tqdm(range(total_pages)):
#             driver.get(url + "?p=" + str(i))
#             contents = driver.find_elements_by_class_name('cat_image-1byrW')
#             for content in contents:
#                 image = content.get_attribute('src')
#                 file.write(image + "\n")
#                 count = count + 1
# file.close()
# print("total count is {}".format(count))

In [22]:
# Going through all the sub categories and saving them as a pickle file containing only the links

for main_category in tqdm(categories):
    main_category_path = base_path + main_category + "/"
    
    if not os.path.exists(main_category_path):
        os.makedirs(main_category_path)
        
    for sub_category in tqdm(categories[main_category]):
        sub_category_path = main_category_path + sub_category
        
        if not os.path.exists(sub_category_path):
            os.makedirs(sub_category_path)
            
        url = base_url + sub_category
        
        html = urlopen(url)
        soup = BeautifulSoup(html, 'lxml')
        
        # now getting the total number of pages for each category from total items, taking 84 items for each page
        total_items = int(soup.find("div", {"class": "cat_count-1zSG1"}).get_text().split(" ")[0].replace(',', ''))
        total_pages = math.ceil(total_items/84)
        
        # now opening each of the pages in selenium because in beautiful soup all the images are not available
        # and putting all the urls to as a list in pickle format
        items = []
        for i in tqdm(range(total_pages)):
            driver.get(url + "?p=" + str(i))
            contents = driver.find_elements_by_class_name('cat_imageLink-OPGGa')
            for content in contents:
                items.append(content.get_attribute('href'))
        
        with open(sub_category_path + "/" + sub_category + ".pkl", "wb") as f:
            pickle.dump(items, f)

In [ ]:
# Now to go through each of the subcategories and saving their description in the following format

for main_category in tqdm(categories):
    main_category_path = base_path + main_category + "/"
    for sub_category in tqdm(categories[main_category]):
        sub_category_path = main_category_path + sub_category
            
        # opening the pickle file containing the links
        with open(sub_category_path + "/" + sub_category + ".pkl", "rb") as f:
            items = pickle.load(f)
            
        item_details = {}
        for item in tqdm(items):
            try:
                desc = {} # dictionary to contain the description of the image

                url = item
                html = urlopen(url)
                soup = BeautifulSoup(html, 'lxml')

                # title, type and price
                try:
                    title_info = soup.find("div", {"class": "h-product-title"})
                    desc['title'] = title_info.find("a").find('h2').get_text()
                    desc['type'] = title_info.find("h1", {"class": "title-typo"}).get_text()

                    desc['price'] = soup.find("div", {"class": "h-product-price"}).find("div").get_text()
                except:
                    pass

                # product description and size
                try:
                    product_info = soup.find("div", {"class": "z-hlwd-ui_tracking-view-wrapper"}).find("div", {"role": "tabpanel"}).find_all("div", {"class": "h-tabs__panel"})

                    product_description_elements = product_info[0].find_all("p", {"class": "detail"})
                    product_description = {}
                    for i in product_description_elements:
                        product_description[i.get_text()] = {}

                        elements = i.findNext('div').find_all('p')
                        for j in elements:
                            product_description[i.get_text()][j.find_all("span")[0].get_text().strip()] = j.find_all("span")[1].get_text().strip()

                    desc["description"] = product_description

                    size_description = {}
                    size_elements = product_info[1].find_all("p")

                    for i in size_elements:
                        spans = i.find_all("span")
                        size_description[spans[0].get_text().strip()] = spans[1].get_text().strip()
                    desc["size"] = size_description
                except:
                    pass

                # product image links
                try:
                    image_src = {"normal_images":[], "product_images":[]}
                    images = soup.find("div", {"id": "topsection-thumbnail-scroller"}).find_all("img")
                    for img in images:
                        link = img['src']
                        if "packshot" in link:
                            image_src["product_images"].append(link.split("?")[0])
                        else:
                            image_src["normal_images"].append(link.split("?")[0])
                    desc["images"] = image_src
                except:
                    pass

                # positive image links
                try:
                    positive_image_links = []
                    positive_images = soup.find("div", {"id": "topsection-color-scroller"}).find_all("a")
                    for img in positive_images[1:]:
                        positive_image_links.append(img["href"])
                    desc["positive_image_links"] = positive_image_links
                except:
                    pass

                # positive image sources
                try:
                    positive_image_src = []
                    positive_images = soup.find("div", {"id": "topsection-color-scroller"}).find_all("img")
                    for img in positive_images[1:]:
                        positive_image_src.append(img['src'].split("?")[0])
                    desc["positive_image_sources"] = positive_image_src

                    item_details[url] = desc
                except:
                    pass
            except:
                pass
            
            
        # saving the description in pickle format
        with open(sub_category_path + "/" + sub_category + "_description.pkl", "wb") as f:
            pickle.dump(item_details, f)



#### code to download all the images

In [2]:
import aria2p

In [3]:
# initialization, these are the default values
aria2 = aria2p.API(
    aria2p.Client(
        host="http://localhost",
        port=6800,
        secret=""
    )
)

In [19]:
# putting all the image sources in a single file 
file = open("images.txt", "w")

for main_category in tqdm(categories):
    main_category_path = base_path + main_category + "/"
    for sub_category in tqdm(categories[main_category]):
        sub_category_path = main_category_path + sub_category
            
        # opening the pickle file containing the description
        with open(sub_category_path + "/" + sub_category + "_description.pkl", "rb") as f:
            items = pickle.load(f)
            
        for item in tqdm(items):
            desc = items[item]
            try:
                normal_images = desc['images']['normal_images']
                product_images = desc['images']['product_images']

                for image in normal_images:
                    file.write(image + "\n")
                for image in product_images:
                    file.write(image + "\n")
            except:
                pass
                
file.close()

In [20]:
# to check total how many images are there
count = 0

for main_category in tqdm(categories):
    main_category_path = base_path + main_category + "/"
    for sub_category in tqdm(categories[main_category]):
        sub_category_path = main_category_path + sub_category
            
        # opening the pickle file containing the description
        with open(sub_category_path + "/" + sub_category + "_description.pkl", "rb") as f:
            items = pickle.load(f)
            
        for item in tqdm(items):
            desc = items[item]
            try:
                normal_images = desc['images']['normal_images']
                product_images = desc['images']['product_images']

                for image in normal_images:
                    count = count + 1
                for image in product_images:
                    count = count + 1
            except:
                pass
                
print(count)


243466


In [21]:
# terminal command for downloading images saved in a text file
#!sudo aria2c -d="./zalando/images" -iimages.txt -j100